In [6]:
import os, sys
sys.path.append(os.path.join("../.."))

import privugger as pv
import numpy as np

In [12]:
#
# Example of a program to analyze
# 
priors = dict.fromkeys(['age', 'height', 'weight'])
alpha = lambda priors:[np.mean(priors['age']), np.mean(priors['height']), np.mean(priors['weight']]

SyntaxError: invalid syntax (<ipython-input-12-63eb9e39623c>, line 5)

In [10]:
#####################################################
######################################################
# Example of how to use privugger on the alpha program 
#########################################################


# Define the prior of the input to alpha
priors             = dict.fromkeys(['age', 'height', 'weight'])

priors['age'] = pv.Normal('age', mu=50, std=1, num_elements=100)
priors['height'] = pv.Normal('height', mu=175, std=1, num_elements=100)
priors['weight'] = pv.Normal('weight', mu=75, std=1, num_elements=100)


#Collect priors in dataset
ds                 = pv.Dataset(input_specs = [priors['age'], priors['height'], priors['weight']])

#Define program which includes, dataset, output type of program and the function itself
program            = pv.Program(dataset=ds, output_type = pv.Float, function=alpha)

#Run inference by providing the program to infer method
trace              = pv.infer(program, cores=2, draws=1000)

#Calculate privacy measure like mutual information (between age and output in this simple case)
#mutual_information = pv.mi_sklearn(trace.posterior.age.values, trace.posterior.output.values.ravel())

SyntaxError: invalid syntax (<unknown>, line 2)

In [15]:
trace

Inference data with groups:
	> posterior
	> sample_stats

In [15]:
d= trace.to_dict()
d['posterior']['age']


/Users/chris/opt/anaconda3/lib/python3.7/site-packages/arviz/data/inference_data.py:437: UserWarning: The attributes are not same for all groups. Considering only the first group `attrs`
  "The attributes are not same for all groups."


array([[[51.98893208, 49.7514477 , 51.17229877, ..., 49.15475973,
         50.00128803, 48.29627133],
        [52.52256213, 50.51051621, 49.83163422, ..., 49.03086769,
         49.34536547, 49.37860814],
        [47.03487528, 49.44836638, 50.00561524, ..., 49.8953771 ,
         50.9698532 , 50.68824397],
        ...,
        [51.9561171 , 49.59760952, 50.40773021, ..., 51.29759713,
         49.41366934, 50.96781229],
        [52.20614928, 49.80486021, 50.5574015 , ..., 48.79678298,
         49.93344297, 48.70350236],
        [51.47620828, 48.87385123, 50.4709316 , ..., 48.48760466,
         50.57984865, 50.03160543]],

       [[50.34043485, 47.54640607, 50.27347567, ..., 51.47138883,
         49.67839972, 49.75242645],
        [51.26461323, 49.90370559, 50.52586998, ..., 50.05053896,
         50.79672157, 49.46695719],
        [48.15879305, 49.97271686, 49.89043467, ..., 49.26310997,
         49.22732579, 49.93566085],
        ...,
        [50.41246359, 50.80227315, 50.14964884, ..., 4

In [ ]:
#####################################################
######################################################
# Example of how to use privugger on the alpha program with a hierarchial model
#########################################################


# Define the prior of the input to alpha and also the hyper parameters 
age_mu             = pv.Uniform('age_mu', lower=1, upper=2, is_hyper_param=True)
age_std            = pv.Normal('age_std', mu=2, std=0.5, is_hyper_param=True)
age                = pv.Normal('age', mu=age_mu, std=age_std, num_elements=1)

#Collect priors in dataset along with hyper parameters
ds                 = pv.Dataset(input_specs = [age_mu, age_std, age])

#Define program which includes, dataset, output type of program and the function itself
program            = pv.Program(dataset=ds, output_type = pv.Float, function=alpha)

#Run inference by providing the program to infer method
trace              = pv.infer(program, cores=2, draws=1000)

#Calculate privacy measure like mutual information (between age and output in this simple case)
mutual_information = pv.mi_sklearn(trace.posterior.age.values, trace.posterior.output.values.ravel())

In [ ]:
trace

In [16]:
######################################################################
####################################################################
# Example of how to use privugger with concatenated priors
#######################################################################


# Define the prior of the input to alpha  
age1               = pv.Normal('age1', mu=0, std=1, num_elements=1)  
age2               = pv.Normal('age2', mu=10,std=1, num_elements=1)
concat             = pv.concatenate(age1,age2, axis=0)

#Collect priors in dataset 
ds                 = pv.Dataset(input_specs = [concat])

#Define program which includes, dataset, output type of program and the function itself
program            = pv.Program(dataset=ds, output_type = pv.Float, function=alpha)

#Run inference by providing the program to infer method
trace              = pv.infer(program, cores=2, draws=1000)

#Calculate privacy measure like mutual information (between age1, age2 and output in this simple case)
mutual_information_age1 = pv.mi_sklearn(trace.posterior.age11.values, trace.posterior.output.values.ravel())
mutual_information_age2 = pv.mi_sklearn(trace.posterior.age22.values, trace.posterior.output.values.ravel())

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [age22, age11]
Could not pickle model, sampling singlethreaded.
Sequential sampling (2 chains in 1 job)
NUTS: [age22, age11]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 6 seconds.


In [17]:
trace

Inference data with groups:
	> posterior
	> sample_stats

In [ ]:
######################################################################
####################################################################
# Example of how to use privugger with stacked priors
#######################################################################


# Define the prior of the input to alpha  
age1               = pv.Normal('age1', mu=0, std=1, num_elements=2)  
age2               = pv.Normal('age2', mu=10,std=1, num_elements=2)
stack              = pv.stack([age1,age2])

#Collect priors in dataset 
ds                 = pv.Dataset(input_specs = [stack])

#Define program which includes, dataset, output type of program and the function itself
program            = pv.Program(dataset=ds, output_type = pv.Float, function=alpha)

#Run inference by providing the program to infer method
trace              = pv.infer(program, cores=2, draws=1000)
#Calculate privacy measure like mutual information (between age1, age2 and output in this simple case)
mutual_information_age1 = pv.mi_sklearn(trace.posterior.age11.values[0], trace.posterior.output.values.ravel())
mutual_information_age2 = pv.mi_sklearn(trace.posterior.age22.values[0], trace.posterior.output.values.ravel())